# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

Matplotlib is building the font cache; this may take a moment.


In [2]:
import sys
sys.executable

'/root/miniconda/bin/python'

In [3]:
%env TARGET=10.10.232.160

env: TARGET=10.10.232.160


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

/bin/bash: docker: command not found


# Initial Nmap:

In [7]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.232.160


In [8]:
!ping -c 5 $TARGET

PING 10.10.232.160 (10.10.232.160) 56(84) bytes of data.
64 bytes from 10.10.232.160: icmp_seq=1 ttl=64 time=16.2 ms
64 bytes from 10.10.232.160: icmp_seq=2 ttl=64 time=0.636 ms
64 bytes from 10.10.232.160: icmp_seq=3 ttl=64 time=13.3 ms
64 bytes from 10.10.232.160: icmp_seq=4 ttl=64 time=1.13 ms
64 bytes from 10.10.232.160: icmp_seq=5 ttl=64 time=0.501 ms

--- 10.10.232.160 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4052ms
rtt min/avg/max/mdev = 0.501/6.351/16.208/6.917 ms


# Quick Scan:

In [9]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-12 07:59 UTC
Initiating ARP Ping Scan at 07:59
Scanning 10.10.232.160 [1 port]
Completed ARP Ping Scan at 07:59, 0.28s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 07:59
Completed Parallel DNS resolution of 1 host. at 07:59, 0.05s elapsed
Initiating SYN Stealth Scan at 07:59
Scanning ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160) [1000 ports]
Discovered open port 21/tcp on 10.10.232.160
Discovered open port 22/tcp on 10.10.232.160
Discovered open port 80/tcp on 10.10.232.160
Completed SYN Stealth Scan at 07:59, 0.44s elapsed (1000 total ports)
Nmap scan report for ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160)
Host is up, received arp-response (0.0016s latency).
Scanned at 2021-10-12 07:59:58 UTC for 1s
Not shown: 997 closed ports
Reason: 997 resets
PORT   STATE SERVICE REASON
21/tcp open  ftp     syn-ack ttl 64
22/tcp open  ssh     syn-ack ttl 64
80/tcp open  http    syn-ack ttl 64

# All Ports:

In [10]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-12 08:00 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 08:00
Scanning 10.10.232.160 [1 port]
Completed ARP Ping Scan at 08:00, 0.33s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 08:00
Completed Parallel DNS resolution of 1 host. at 08:00, 0.01s elapsed
Initiating SYN Stealth Scan at 08:00
Scanning ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160) [65535 ports]
Discovered open port 22/tcp on 10.10.232.160
Discovered open port 21/tcp on 10.10.232.160
Discovered open port 80/tcp on 10.10.232.160
Completed SYN Stealth Scan at 08:00, 6.24s elapsed (65535 total ports)
Initiating Service scan at 08:00
Scanning 3 services on ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160)
Completed Service scan at 08:00, 6.03s elapsed (3 services on 1 host)
NSE: Script scanning 10.10.232.160.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 08:00
Completed NSE at 08:00, 0.07s elaps

In [11]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [12]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=21,22,80


# No Ping:

In [13]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-12 08:00 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 08:00
Scanning 10.10.232.160 [1 port]
Completed ARP Ping Scan at 08:00, 0.03s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 08:00
Completed Parallel DNS resolution of 1 host. at 08:00, 0.00s elapsed
Initiating SYN Stealth Scan at 08:00
Scanning ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160) [1000 ports]
Discovered open port 80/tcp on 10.10.232.160
Discovered open port 21/tcp on 10.10.232.160
Discovered open port 22/tcp on 10.10.232.160
Completed SYN Stealth Scan at 08:00, 0.09s elapsed (1000 total ports)
Initiating Service scan at 08:00
Scanning 3 services on ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160)
Completed Service scan at 08:00, 6.02s elapsed (3 services on 1 host)
NSE: Script scanning 10.10.232.160.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 08:00
Completed NSE at 08:00, 0.02s elapsed

In [14]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [15]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-12 08:00 UTC
NSE: Loaded 121 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 08:00
Completed NSE at 08:00, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 08:00
Completed NSE at 08:00, 0.00s elapsed
Initiating ARP Ping Scan at 08:00
Scanning 10.10.232.160 [1 port]
Completed ARP Ping Scan at 08:00, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 08:00
Completed Parallel DNS resolution of 1 host. at 08:00, 0.00s elapsed
Initiating SYN Stealth Scan at 08:00
Scanning ip-10-10-232-160.eu-west-1.compute.internal (10.10.232.160) [3 ports]
Discovered open port 22/tcp on 10.10.232.160
Discovered open port 80/tcp on 10.10.232.160
Discovered open port 21/tcp on 10.10.232.160
Completed SYN Stealth Scan at 08:00, 0.04s elapsed (3 total ports)
NSE: Script scanning 10.10.232.160.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 08:00
Co

In [16]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-12 08:00 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 08:00
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
too short
NSE Timing: About 97.70% done; ETC: 08:01 (0:00:01 remaining)
Completed NSE at 08:01, 40.00s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 08:01
Completed NSE at 08:01, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan

In [ ]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-12 08:03 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 08:04
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
too short
NSE Timing: About 96.55% done; ETC: 08:04 (0:00:01 remaining)
Completed NSE at 08:04, 40.01s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 08:04
Completed NSE at 08:04, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt